##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite 모델 분석기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/guide/model_analyzer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

TensorFlow Lite Model Analyzer API는 모델의 구조를 나열하여 TensorFlow Lite 형식의 모델을 분석하는 데 도움을 줍니다.


## Model Analyzer API

TensorFlow Lite 모델 분석기에 다음 API를 사용할 수 있습니다.

```
tf.lite.experimental.Analyzer.analyze(model_path=None,
                                      model_content=None,
                                      gpu_compatibility=False)
```

API 세부 정보는 https://www.tensorflow.org/api_docs/python/tf/lite/experimental/Analyzer에서 찾아보거나 Python 터미널에서 `help(tf.lite.experimental.Analyzer.analyze)`를 실행할 수 있습니다.


## 간단한 Keras 모델의 기본 사용법

다음 코드는 모델 분석기의 기본 사용법을 보여줍니다. TFLite 모델 콘텐츠에서 변환된 Keras 모델의 콘텐츠를 플랫 버퍼 객체로 형식화하여 보여줍니다.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)


## MobileNetV3Large Keras 모델의 기본 사용법

이 API는 MobileNetV3Large와 같은 대형 모델에서 효과가 있습니다. 출력이 크기 때문에 주로 이용하는 텍스트 편집기로 찾아볼 수 있습니다.

In [ ]:
model = tf.keras.applications.MobileNetV3Large()
fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)

## GPU 대리자 호환성 확인

ModelAnalyzer API는 <code>gpu_compatibility=True</code> 옵션을 제공하여 주어진 모델의 <a>GPU 대리자</a> 호환성을 확인하는 방법을 제공합니다.


### 사례 1: 모델이 호환되지 않는 경우

다음 코드는 GPU 대리자와 호환되지 않는 2D 텐서 및 `tf.slice`와 함께 `tf.cosh`를 사용하는 간단한 tf.function에 대해 `gpu_compatibility=True` 옵션을 사용하는 방법을 보여줍니다.

호환성 문제가 있는 모든 노드마다 `GPU COMPATIBILITY WARNING`가 표시됩니다.

In [ ]:
import tensorflow as tf

@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  return tf.cosh(x) + tf.slice(x, [1, 1], [1, 1])

converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [func.get_concrete_function()], func)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
fb_model = converter.convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

### 사례 2: 모델이 호환되는 경우

이 예에서 주어진 모델은 GPU 대리자와 호환됩니다.

**참고:** 도구가 호환성 문제를 찾지 못하더라도 모델이 모든 장치에서 GPU 대리자와 잘 작동한다는 보장은 없습니다. 대상 OpenGL 백엔드에서 `CL_DEVICE_IMAGE_SUPPORT` 요소 누락과 같은 런타임 비호환성이 발생할 수 있습니다.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)